In [1]:
import pandas as pd
import anthropic

In [2]:
MODELS = dict(haiku="claude-3-haiku-20240307", sonnet="claude-3-5-sonnet-20240620")

In [3]:
df = pd.read_csv("data/train.csv", index_col=0)

In [4]:
df.iloc[0,0]

'Find the value of the expression $\\dfrac{17}{5} :\\dfrac{34}{3} +1.3$.'

In [5]:
prompt = df.iloc[0,0]

In [6]:
client = anthropic.Anthropic()


## First prompt

In [7]:
response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens = 500,
    messages=[
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": prompt
            }
          ]
        }
  ]
)


In [8]:
print(response.content[0].text)

To solve this expression, we need to follow the order of operations: PEMDAS (Parentheses, Exponents, Multiplication and Division, Addition and Subtraction).

Given expression: $\dfrac{17}{5} :\dfrac{34}{3} +1.3$

Step 1: Perform the division operation.
$\dfrac{17}{5} :\dfrac{34}{3} = \dfrac{17}{5} \times \dfrac{3}{34} = \dfrac{51}{170}$

Step 2: Add the constant 1.3.
$\dfrac{51}{170} + 1.3 = \dfrac{51}{170} + \dfrac{130}{100} = \dfrac{181}{170} + \dfrac{130}{100} = \dfrac{181 \times 100 + 130 \times 170}{170 \times 100} = \dfrac{18100 + 22100}{17000} = \dfrac{40200}{17000} = \dfrac{402}{170}$

Therefore, the value of the expression $\dfrac{17}{5} :\dfrac{34}{3} +1.3$ is $\dfrac{402}{170}$.


## Organize the code and write first prompt

In [9]:
def get_answer_1(task, model_name = MODELS['sonnet']): 
    system = "you are assistant that give correct answers to math tasks, you should only write the numeric answers, or none if you don't know. You should not write ane explanations"
    
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": task
                }
            ]
        }
    ]

    response = client.messages.create( 
        model=model_name,
        max_tokens=10,
        system=system,
        messages=messages
    )
    return(response.content[0].text)

In [25]:
task, answer = df.iloc[0,]

In [26]:
quess = get_answer_1(task)

In [28]:
print(task)
print(f"Quess = {quess}")
print(f"Answer = {answer}")

Find the value of the expression $\dfrac{17}{5} :\dfrac{34}{3} +1.3$.
Quess = 0.5
Answer = 1.6


In [31]:
quess = get_answer_1(task, model_name=MODELS['haiku'])

In [32]:
print(task)
print(f"Quess = {quess}")
print(f"Answer = {answer}")

Find the value of the expression $\dfrac{17}{5} :\dfrac{34}{3} +1.3$.
Quess = 2.1
Answer = 1.6


In [38]:
from tqdm import tqdm

In [39]:
answers = []
quesses = []
for i in tqdm(range(10)):
    task, answer = df.iloc[i,]
    quess = get_answer_1(task)
    answers.append(answer)
    quesses.append(quess)

100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:10<00:00,  1.06s/it]


In [45]:
from pprint import pprint

In [46]:
pprint(list(zip(answers, quesses)))

[('1.6', '0.5'),
 ('24', '24'),
 ('32', '32'),
 ('256', '256'),
 ('53', '346'),
 ('3', '4'),
 ('27000; 64000', '1728'),
 ('2', '2'),
 ('0.1', '0.1'),
 ('1.2', '\\(\\frac{6}{\\sqrt{')]


In [47]:
print(f"Percent of right answers: {mean(answers == quesses}")

SyntaxError: f-string: closing parenthesis '}' does not match opening parenthesis '(' (105316046.py, line 1)

In [57]:
scoreing = sum(x == y for x, y in zip(answers, quesses)) / len(answers)
print(f"the score is {scoreing}")

the score is 0.5


In [58]:
answers = []
quesses = []
for i in tqdm(range(10)):
    task, answer = df.iloc[i,]
    quess = get_answer_1(task, MODELS['haiku'])
    answers.append(answer)
    quesses.append(quess)

100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.16it/s]


In [59]:
scoreing = sum(x == y for x, y in zip(answers, quesses)) / len(answers)
print(f"the score is {scoreing}")

the score is 0.2


## Try to sumbit dummy answer

In [16]:
test_df = pd.read_csv("data/test.csv", index_col=0)

In [17]:
test_df.head()

,problem_text
problem_id,
11919,Find the value of the expression \(4.8 \cdot 2...
8513,The plane's navigation system informs the pass...
7887,The volume of a rectangular parallelepiped is ...
5272,Find the root of the equation: $\left(\dfrac{1...
8295,"At the school, there are two-person camping te..."


In [18]:
def dummy_answer(question_text):
    return 0

In [19]:
answers = test_df.apply(lambda row: dummy_answer(row['problem_text']), axis=1)

In [20]:
answers

problem_id
11919    0
8513     0
7887     0
5272     0
8295     0
        ..
3519     0
7934     0
9390     0
7137     0
5914     0
Length: 100, dtype: int64

In [42]:
answer_df = answers.reset_index()
answer_df.columns = ["problem_id", "answer"]

In [43]:
answer_df.to_csv("data/my_sumbit.csv")

In [50]:
def make_submit(answers: pd.Series, filename="data/my_submit.csv"):
    answer_df = answers.reset_index()
    answer_df.columns = ["problem_id", "answer"]
    answer_df.to_csv(filename, index=False)
    

In [51]:
make_submit(answers)

# Try claude sonnet decision

In [52]:
answers = test_df.apply(lambda row: get_answer_1(row['problem_text']), axis=1)

In [53]:
make_submit(answers)